# Creating a graph for a given streetmap picture using pixels as coordinates

This notebook is an easy way to use OpenCV, NetworkX and OSMNX to create and graph overlaying a picture of a streetmap

## First Part

First load the picture currently named *VideoCapture.png*. Each point you click on will be saved as a node in a NetworkX graph. The pixel coordinates will be saved as node attributes *x* and *y*. When you are done just press ESC and the window showing the picture should close automaticly 

In [1]:
import cv2
import numpy as np
import pandas as pd
import networkx as nx
import osmnx as ox

#This will display all the available mouse click events  
events = [i for i in dir(cv2) if 'EVENT' in i]
print(events)

#This variable we use to store the pixel location
refPt = []
refPt_zwei = []

first_pd = None
second_pd = None
first_id = 1
second_id = 1
G = nx.Graph()
#click event function
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(x,",",y)
        refPt.append([x,y])
        index = len(refPt)
        font = cv2.FONT_HERSHEY_SIMPLEX
        strXY = str(x)+", "+str(y)
        #row = [first_id,x,y]
        #cv2.putText(img, strXY, (x,y), font, 0.5, (255,255,0), 2)
        cv2.putText(img, str(index), (x,y), font, 0.5, (255,255,0), 2)
        #cv2.putText(img_zwei, str(first_id), (x,y), font, 0.5, (255,255,0), 2)
        #Spaltennamen = ['ID','x','y']
        #NeueZeile=pd.DataFrame([row], columns=Spaltennamen)
        #first_id = first_id+1
        #if first_pd is None:
        #    first_pd = NeueZeile
        #else:
        #    first_pd = first_pd.append(NeueZeile)
        #first_pd.to_csv("Points.csv",index=False)
        G.add_node(index)
        G.nodes[index]['x'] = x
        G.nodes[index]['y'] = y
        cv2.imshow("image", img)



#Here, you need to change the image name and it's path according to your directory
img = cv2.imread("VideoCapture.png")
img_zwei = img
img_drei = img
print(img.shape)
cv2.imshow("image", img)

#calling the mouse click event
cv2.setMouseCallback("image", click_event)

cv2.waitKey(0)
cv2.destroyAllWindows()

['EVENT_FLAG_ALTKEY', 'EVENT_FLAG_CTRLKEY', 'EVENT_FLAG_LBUTTON', 'EVENT_FLAG_MBUTTON', 'EVENT_FLAG_RBUTTON', 'EVENT_FLAG_SHIFTKEY', 'EVENT_LBUTTONDBLCLK', 'EVENT_LBUTTONDOWN', 'EVENT_LBUTTONUP', 'EVENT_MBUTTONDBLCLK', 'EVENT_MBUTTONDOWN', 'EVENT_MBUTTONUP', 'EVENT_MOUSEHWHEEL', 'EVENT_MOUSEMOVE', 'EVENT_MOUSEWHEEL', 'EVENT_RBUTTONDBLCLK', 'EVENT_RBUTTONDOWN', 'EVENT_RBUTTONUP']
(1080, 1440, 3)
562 , 639
774 , 639
984 , 637
1054 , 634
1054 , 559
1051 , 362
1010 , 303
954 , 267
773 , 263
575 , 270
502 , 274
496 , 344
491 , 447
489 , 564
491 , 636
492 , 712
475 , 788
418 , 813
359 , 780
335 , 719
332 , 633
407 , 635
331 , 475
338 , 373
367 , 309
423 , 283
1052 , 708
1071 , 787
1127 , 806
1120 , 632
1186 , 607
1207 , 541


## Second Part

Now that all nodes are given, click near a node to capture that node. Next click near another node that you want to connect with the first node with an edge. The edge will be saved automaticly, a line between those nodes will be drawn, and the distance between those  nodes will be saved as *distance* in an edge attribute. If you clicked on the wrong first node, just use the right mouse button to undo your previous choice. Only works for the first node of each edge. Once the second node is chosen, an edge will be saved, which can not be undone.
All eaches made will be printed out below. When finished, just press escape.

In [2]:
import math
first_pd = None
second_pd = None
Spaltennamen = ['ID','x','y']
img = cv2.imread("VideoCapture.png")
img_zwei = img
img_drei = img
for i in range(0,len(refPt)):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img_zwei, str(i+1), (refPt[i][0],refPt[i][1]), font, 0.5, (255,255,0), 2)
    row = [i,refPt[i][0],refPt[i][1]]
    NeueZeile=pd.DataFrame([row], columns=Spaltennamen)
    if first_pd is None:
        first_pd = NeueZeile
    else:
        first_pd = first_pd.append(NeueZeile)
first_pd.to_csv("Points.csv",index=False)
node_list = []

def get_distance_between_points(a_x,a_y,b_x,b_y):
    delta_x = abs(b_x-a_x)
    delta_y = abs(b_y-a_y)
    return math.sqrt(math.pow(delta_x,2)+math.pow(delta_y,2))

def click_event_new(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        font = cv2.FONT_HERSHEY_SIMPLEX
        if len(node_list)==2:
            node_list.clear()
        
        node = ox.get_nearest_node(G,(y,x),method='euclidean')
        print(node)
        node_list.append(node)
        if len(node_list)==2:
            G.add_edge(node_list[0],node_list[1])
            dist = get_distance_between_points(G.nodes[node_list[0]]['x'],G.nodes[node_list[0]]['y'],G.nodes[node_list[1]]['x'],G.nodes[node_list[1]]['y'])
            G[node_list[0]][node_list[1]]['distance'] = dist
            print("edge added between {} and {}".format(node_list[0],node_list[1]))
        img = cv2.imread("VideoCapture.png")
        for i in range(0,len(refPt)):
            if (i+1) not in node_list:
                cv2.putText(img, str(i+1), (refPt[i][0],refPt[i][1]), font, 0.5, (255,255,0), 2)
            else:
                cv2.putText(img, str(i+1), (refPt[i][0],refPt[i][1]), font, 0.5, (255,0,0), 2)
        for edge in G.edges:
            cv2.line(img,(G.nodes[edge[0]]['x'],G.nodes[edge[0]]['y']),(G.nodes[edge[1]]['x'],G.nodes[edge[1]]['y']),(255,0,0),3)
        cv2.imshow("image", img)
        
    if event == cv2.EVENT_RBUTTONDOWN:
        #print(x,",",y)
        node_list.clear()
        font = cv2.FONT_HERSHEY_SIMPLEX
        img = cv2.imread("VideoCapture.png")
        for i in range(0,len(refPt)):
            if i not in node_list:
                cv2.putText(img, str(i+1), (refPt[i][0],refPt[i][1]), font, 0.5, (255,255,0), 2)
            else:
                cv2.putText(img, str(i+1), (refPt[i][0],refPt[i][1]), font, 0.5, (255,0,0), 2)
        for edge in G.edges:
            cv2.line(img,(G.nodes[edge[0]]['x'],G.nodes[edge[0]]['y']),(G.nodes[edge[1]]['x'],G.nodes[edge[1]]['y']),(255,0,0),3)
        cv2.imshow("image", img)
        




In [3]:

cv2.imshow("image", img)

#calling the mouse click event
cv2.setMouseCallback("image", click_event_new)

cv2.waitKey(0)
cv2.destroyAllWindows()

1
2
edge added between 1 and 2
2
3
edge added between 2 and 3
3
4
edge added between 3 and 4
4
5
edge added between 4 and 5
4
30
edge added between 4 and 30
4
27
edge added between 4 and 27
30
31
edge added between 30 and 31
31
32
edge added between 31 and 32
27
28
edge added between 27 and 28
28
29
edge added between 28 and 29
5
6
edge added between 5 and 6
6
7
edge added between 6 and 7
7
8
edge added between 7 and 8
8
9
edge added between 8 and 9
9
10
edge added between 9 and 10
10
11
edge added between 10 and 11
11
12
edge added between 11 and 12
12
13
edge added between 12 and 13
11
26
edge added between 11 and 26
13
14
edge added between 13 and 14
14
15
edge added between 14 and 15
15
16
edge added between 15 and 16
16
17
edge added between 16 and 17
17
18
edge added between 17 and 18
18
19
edge added between 18 and 19
19
20
edge added between 19 and 20
20
21
edge added between 20 and 21
21
22
edge added between 21 and 22
22
15
edge added between 22 and 15
15
1
edge added between

## Last Part
Save graph with all attributes as graphml file

In [4]:
nx.write_graphml(G,'third_try.graphml')